This portion of the analysis will focus on private planes. We define a private plane as one which carries less than 20 passengers total. Private planes generally have either 1 or 2 engines. Our goal will be to determine the safest make and model of private planes, as well as whether it is safer to have 1 or 2 engines.

We begin by importing and filtering the data. We filtered it to look only at US flights which occurred after the creation of the TSA on November 19, 2001. We also removed several columns which were not necessary to our analysis.

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv('../data/Aviation_Data.csv', encoding="latin-1", low_memory=False)#, parse_dates=['Event.Date'])

In [10]:
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      90348 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

In [23]:
df = df.rename(columns={c: c.title().replace('.', '_') for c in df.columns})
usa=df.loc[(df['Country']=='United States') & (df['Event_Date']>='2001-11-19')]
usa['Event_Date'] = pd.to_datetime(usa['Event_Date'])

<ipython-input-23-59f51f8b9c23>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa['Event_Date'] = pd.to_datetime(usa['Event_Date'])


Next, we will clean up some of the columns which have missing values. We will begin with the columns containing data on injuries and fatalities. From examining the "Injury Severity" column we can see that all the missing values in the "Total_Fatal_Injuries" column correspond to a value of "Non-Fatal" in "Injury_Severity". Therefore we will see these missing values equal to zero in the "Total_Fatal_Injuries" column.

In [28]:
usa.loc[usa['Total_Fatal_Injuries'].isna(), 'Total_Fatal_Injuries']=0

c:\Users\jimmy\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


The "Total.Uninjured" column contains missing values. This column will be important, as we need to know the number of passengers to distinguish private from commercial flights. For this column we will impute the median value, which is 1, into the null values.

There are two other columns containing data on serious and minor injuries. Many of these contain missing values too. Our approach to this will be to look at the ratio of minor/serious injuries in the dataset to the total number of uninjured patients in the dataset. We will use this ratio to impute missing values.

In [48]:
minor_injuries=usa.loc[(usa['Total_Uninjured'].notna()) & (usa['Total_Minor_Injuries'].notna())]
minor_injuries_ratio=minor_injuries['Total_Minor_Injuries'].sum()/minor_injuries['Total_Uninjured'].sum()

serious_injuries=usa.loc[(usa['Total_Uninjured'].notna()) & (usa['Total_Serious_Injuries'].notna())]
serious_injuries_ratio=serious_injuries['Total_Serious_Injuries'].sum()/serious_injuries['Total_Uninjured'].sum()

usa.loc[usa['Total_Uninjured'].isna(), 'Total_Uninjured']=usa['Total_Uninjured'].median()
usa.loc[usa['Total_Minor_Injuries'].isna(), 'Total_Minor_Injuries']=round(minor_injuries_ratio*usa['Total_Uninjured'], 0)
usa.loc[usa['Total_Serious_Injuries'].isna(), 'Total_Serious_Injuries']=round(serious_injuries_ratio*usa['Total_Uninjured'], 0)


Now that we have filled in the missing values for the injuries and fatalities columns, we can add up the values of all these columns to get the number of passengers. Doing so will allow us to distinguish private planes vs commercial ones.

In [52]:
usa['Passengers']=usa['Total_Uninjured']+usa['Total_Minor_Injuries']+usa['Total_Serious_Injuries']+usa['Total_Fatal_Injuries']

<ipython-input-52-fad06e55e0b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa['Passengers']=usa['Total_Uninjured']+usa['Total_Minor_Injuries']+usa['Total_Serious_Injuries']+usa['Total_Fatal_Injuries']


In [57]:
private_planes=usa.loc[usa['Passengers']<20]

In [58]:
private_planes

,Event_Id,Investigation_Type,Accident_Number,Event_Date,Location,Country,Latitude,Longitude,Airport_Code,Airport_Name,...,Air_Carrier,Total_Fatal_Injuries,Total_Serious_Injuries,Total_Minor_Injuries,Total_Uninjured,Weather_Condition,Broad_Phase_Of_Flight,Report_Status,Publication_Date,Passengers
51801,20011128X02307,Accident,FTW02FA037,2001-11-19,"PATTERSON, LA",United States,29.713333,-91.331945,PTN,Harry P. Williams Memorial,...,NaN,1.0,0.0,0.0,1.0,VMC,Takeoff,Probable Cause,30-05-2003,2.0
51802,20011129X02313,Accident,MIA02LA035,2001-11-20,"FAIRHOPE, AL",United States,30.461667,-87.9,AL78,Klumpp Field,...,NaN,0.0,0.0,0.0,2.0,IMC,Landing,Probable Cause,20-02-2002,2.0
51803,20011130X02325,Accident,MIA02LA028,2001-11-20,"STUART, FL",United States,27.083056,-80.341111,NaN,NaN,...,NaN,0.0,0.0,1.0,1.0,VMC,Approach,Probable Cause,28-05-2002,2.0
51804,20011203X02331,Accident,MIA02LA025,2001-11-20,"EVANS, GA",United States,33.371945,-82,NaN,NaN,...,NaN,0.0,1.0,0.0,1.0,VMC,Cruise,Probable Cause,01-04-2003,2.0
51805,20011127X02302,Accident,MIA02LA024,2001-11-20,"CRYSTAL RIVER, FL",United States,NaN,NaN,CGC,Crystal River Airport,...,NaN,0.0,0.0,0.0,1.0,VMC,Taxi,Probable Cause,05-02-2002,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90343,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,...,NaN,0.0,1.0,0.0,0.0,NaN,NaN,NaN,29-12-2022,1.0
90344,20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0
90345,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,...,NaN,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022,1.0
90346,20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,...,MC CESSNA 210N LLC,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0
